# Description of Notebook
- This notebook extracts data from SF Gov, does spatial joins as needed, makes a few data type adjustment, and loads the data into a local Postgres database for further transformations and hosting for the final analysis. 
- The data pulled includes active parcels, fire incidents, fire inspections, fire inspection violations, DBI inspection complaints, DBI violations, and Tax Assessor Information (2014) to provide additional metadata.
- Our main goal for this project is to help the SFFD and DBI prioritize their R-2 property inspections which are required annually and at least every five years respectively. 
- The goal of prioriziation is to identify the highest risk buildings and prioritize those first.
- Functions are included in the module/pipeline_module.py file

## Next Steps For Notebook
- Clean up code and functionalize more in this notebook to improve readability and repeatability.

In [10]:
import sys
import os
# Get the current working directory
cwd = os.getcwd()
# Navigate up one level in the directory hierarchy
parent_dir = os.path.abspath(os.path.join(cwd, '..'))
relative_path = os.path.join(parent_dir+'/modules/')
sys.path.append(relative_path)

# Import Dependencies

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pipeline_module import pull_opensf_data
from pipeline_module import extract_coordinates
from pipeline_module import shape_extract
from pipeline_module import spatial_join
from pipeline_module import upload_to_postgres
import psycopg2
from sqlalchemy import create_engine
import ast
import json
import geopandas as gpd
from shapely.geometry import shape
from shapely.geometry import Point
from dotenv import load_dotenv

# Import Data from API

## Define common variables for all API pulls
- This includes the site and required keys.
- Additional users would need to create a .env file in their cloned repo and add their unique keys.

In [2]:
#website for Socrata API 
site = 'data.sfgov.org'

#Credentials loaded from .env file, new users of this notebook must create their own credentials and .env file
#after cloning the gitrepo
load_dotenv()
app_token = os.getenv('SFDATA_APP_TOKEN')
api_key_id = os.getenv('SFDATA_KEY_ID')
api_secret_key = os.getenv('SFDATA_SECRET_KEY')
postgres_connstr = os.getenv('POST_CONN_STR')

## Import Parcel Data
- Some of the data sets we're working with do not have parcel number included (e.g. fire incident/inspection reports). We will need to spatially join those points with the parcel polygons so we have parcel number included with the records for subsequent joins to assessor data for additional metadata.
- We will also join to DBI/HIS inspection data to confirm our spatial join method against block/lot in those datasets.
- We have started with just active parcels. However, we could go back in the future and incorporate a time element to our joins for improved accuracy.

In [3]:
#Pull parcel data from the API 
parcel_endpoint = 'acdm-wktn'
pulltype = 'all' #pull all parcels available in dataset
filters = 'active=True' #filter so that we're only pulling active parcels. 
                        #It could be more accurate to pull historical parcels and match on year but this is for first pass analysis.

parcels_df = pull_opensf_data(site, 
                              parcel_endpoint, 
                              app_token, 
                              api_key_id, 
                              api_secret_key,
                              pulltype,
                              filters)

In [4]:
#Perform initial transformations on parcels dataframe and get it ready for spatial joins
final_parcels_df = parcels_df.rename(columns={'mapblklot':'parcel_number'}) #rename for clarity and later joins
final_parcels_df = final_parcels_df[['parcel_number','shape']] #Drop other columns as we're just using this to match parcel_number with points
final_parcels_df['geometry'] = final_parcels_df['shape'].apply(shape_extract) #convert to shape using shapely for spatial joins 

In [5]:
final_parcels_df = final_parcels_df.drop(['shape'],axis=1).drop_duplicates() #drop duplicates from other columns that aren't parcel and polygon

## Import Fire Incidents Data
- Import fire incidents from API and join with final_parcels_df as no parcel number was found in data
- Perform data preprocessing to ensure database contains proper datatypes and no formats that cannot be stored. 
- Export results to Postgres database for further processing (transformations) in SQL.

In [6]:
fire_incidents_endpoint = 'wr8u-xric' #Fire incidents endpoint
pulltype = 'all' #pull all parcels available in dataset
filters = 'area_of_fire_origin IS NOT NULL' #This filter removes incidents that are not related to fire (e.g. EMS, etc.)
fire_incidents_df = pull_opensf_data(site, 
                                     fire_incidents_endpoint, 
                                     app_token, 
                                     api_key_id, 
                                     api_secret_key,
                                     pulltype,
                                     filters)

In [7]:
#Preprocessing to make sure our date types transfer correctly
fire_incidents_df['incident_number'] = fire_incidents_df['incident_number'].astype(int)
fire_incidents_df['exposure_number'] = fire_incidents_df['exposure_number'].astype(int)
fire_incidents_df['exposure_number'] = fire_incidents_df['exposure_number'].astype(str)
fire_incidents_df['address'] = fire_incidents_df['address'].astype(str)
fire_incidents_df['incident_date'] = pd.to_datetime(fire_incidents_df['incident_date'])
fire_incidents_df['alarm_dttm'] = pd.to_datetime(fire_incidents_df['alarm_dttm'])
fire_incidents_df['arrival_dttm'] = pd.to_datetime(fire_incidents_df['arrival_dttm'])
fire_incidents_df['close_dttm'] = pd.to_datetime(fire_incidents_df['close_dttm'])

In [8]:
spatial_txt = 'point'
fire_incidents_df, fire_incidents_parcel_df = spatial_join(fire_incidents_df, 
                                                           spatial_txt, 
                                                           final_parcels_df)

In [9]:
#Check if we're duplicating data. This is looking OK, a few duplicates but nothing to worry too much about at this point
#There are a few points mapping to multiple parcels but we can take care of that later. If this was a longer project
#Could do some additional spatial logic to make sure mapping was only to one
print(fire_incidents_df['incident_number'].count())
print(fire_incidents_df['incident_number'].nunique())
print(fire_incidents_parcel_df['incident_number'].count())
print(fire_incidents_parcel_df['incident_number'].nunique())

36806
36640
36891
36640


In [10]:
'''
There are a lot of incidents that do not match a parcel. Based on a review of these incidents these occur at intersections
primarily and are not related to building fires (e.g. car fire, etc.). Therefore, we can drop these at this point as
they do not pertain to our question of R-2 building and fire inspections
'''
fire_incidents_parcel_df['parcel_number'].nunique() 

6393

In [11]:
#per comment above, remove fire incidents not linked with parcel as these are not building related (unless tagged wrong but let's assume this for now)
#drop geometry columns for now as Postgres can't handle and we've already matched to parcel which is our join key moving forward
fire_incidents_final = fire_incidents_parcel_df[~fire_incidents_parcel_df['parcel_number'].isna()]
fire_incidents_final = fire_incidents_final.drop(['index_right','point','geometry'],axis=1).drop_duplicates()

In [12]:
tablename = 'fire_incidents_test'
upload_to_postgres(fire_incidents_final, postgres_connstr, tablename)

Table uploaded


## Import Assessor Data
- We need to link parcels with the appropriate property metadata using parcel number as a join key.
- Using the spatial joins, all of our fire/dbi datasets will have parcel number associated with the record.
- Assessor data has important metadata (e.g. construction date) which could help us determine high risk properties.
- Take 2014 as it is most accurate (per problem statement).
- Upload to Postgres for additional transformations.

In [3]:
filters = "closed_roll_year=2014" #per the problem statement 2014 data is most accurate so we will use that
pulltype = 'all'
assessor_endpoint = 'wv5m-vpq2'
assessor_df = pull_opensf_data(site, 
                              assessor_endpoint, 
                              app_token, 
                              api_key_id, 
                              api_secret_key,
                              pulltype,
                              filters)

In [9]:
#break apart lat/long so we can upload and use to count distinct parcels on the same area
#we need to do this for condo's to get the true number of units on the same location
#This method ended up not being used for the final analysis but helped in our understanding of the records and could 
#be useful in future analyses.
key_col = 'parcel_number'
col_name = 'the_geom'
int_assessor_df = extract_coordinates(assessor_df,
                                      key_col,
                                      col_name)

In [21]:
#Preprocessing to make sure our date types transfer correctly
int_assessor_df['number_of_units'] = int_assessor_df['number_of_units'].astype(float).astype(int)

In [23]:
#Upload data to postgres database (running locally) to create extract and allow some joins to be done in SQL
tablename = 'assessor_raw'
upload_to_postgres(int_assessor_df, postgres_connstr, tablename)

816

## Import Fire Inspections
- Pull fire inspections and spatially join with parcels to add parcel_number to dataframe
- The inspections include violations but violations table has additional information. Join in SQL.

In [13]:
pulltype = 'all'
fire_inspections_endpoint = 'wb4c-6hwj'
fire_inspec_df = pull_opensf_data(site, 
                              fire_inspections_endpoint, 
                              app_token, 
                              api_key_id, 
                              api_secret_key,
                              pulltype)

In [17]:
#Spatial join, although dbi inspections actually has block number and lot number (parcel number when combined)
spatial_txt = 'location'
fire_inspec_df, fire_inspec_parcel_df = spatial_join(fire_inspec_df, 
                                                           spatial_txt, 
                                                           final_parcels_df)

In [18]:
fire_inspec_parcel_df.head()

,inspection_number,inspection_type,inspection_type_description,address,zipcode,battalion,station,bfp_district,billable_inspection,inspection_start_date,...,referral_number,interest_amount,penalty_amount,posting_fee,corrective_action_date,referral_agency,lien_date,geometry,index_right,parcel_number
0,14575,04,Complaint Inspection,832 Folsom St,94103,03,01,03S,False,2005-09-23T00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-122.40232 37.78158),136715.0,3733017
1,493175,53,Entertainment Com. Ref. Insp.,494 - 498 14th St,94103,02,36,02S,False,2023-01-11T00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-122.42417 37.76822),121386.0,3533026
2,85368,05,Permit Approval Inspection,698 02nd St,94107,03,08,03S,False,2010-07-15T00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-122.39105 37.78055),148343.0,3788006
3,93797,02,Follow Up Inspection,698 02nd St,94107,03,08,03S,False,2010-11-19T00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-122.39105 37.78055),148343.0,3788006
4,93873,01,Initial Inspection,698 02nd St,94107,03,08,03S,False,2010-11-19T00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-122.39105 37.78055),148343.0,3788006


In [19]:
#Check if we're duplicating data. This is looking OK, a few duplicates but nothing to worry too much about at this point
#There are a few points mapping to multiple parcels but we can take care of that later. If this was a longer project
#Could do some additional spatial logic to make sure mapping was only to one
print(fire_inspec_df['inspection_number'].count())
print(fire_inspec_df['inspection_number'].nunique())
print(fire_inspec_parcel_df['inspection_number'].count())
print(fire_inspec_parcel_df['inspection_number'].nunique())

369743
369257
378382
369257


In [48]:
# We're missing about 5% in the spatially join but at this level of analysis I think it is acceptable.
# If time permits, I will circle back and see why
fire_inspec_parcel_df[fire_inspec_parcel_df['parcel_number'].isna()]['inspection_number'].count()

13525

In [49]:
#Drop these fields as postgres won't like and we have parcel now. 
fire_inspec_parcel_df = fire_inspec_parcel_df.drop(['location','geometry'],axis=1) 

In [79]:
#Preprocessing to make sure our date types transfer correctly
fire_inspec_parcel_df['inspection_start_date'] = pd.to_datetime(fire_inspec_parcel_df['inspection_start_date'], errors='coerce')
fire_inspec_parcel_df['inspection_end_date'] = pd.to_datetime(fire_inspec_parcel_df['inspection_end_date'], errors='coerce')
fire_inspec_parcel_df['return_date'] = pd.to_datetime(fire_inspec_parcel_df['return_date'], errors='coerce')

In [80]:
# Drop columns containing "@" in their names
columns_to_drop = fire_inspec_parcel_df.filter(regex='@', axis=1).columns
fire_inspec_parcel_df.drop(columns=columns_to_drop, inplace=True)

In [84]:
#Upload data to postgres database (running locally) to create extract and allow some joins to be done in SQL
tablename = 'fire_inspection_raw'
upload_to_postgres(fire_inspec_parcel_df, postgres_connstr, tablename)

350

## Fire Violations
- Pull down the fire violations dataset using the API.
- Spatially join with parcels.
- Upload to Postgres Database.

In [20]:
pulltype = 'all'
fire_violations_endpoint = '4zuq-2cbe'
fire_vio_df = pull_opensf_data(site, 
                              fire_violations_endpoint, 
                              app_token, 
                              api_key_id, 
                              api_secret_key,
                              pulltype)

In [21]:
#Spatial join, although dbi inspections actually has block number and lot number (parcel number when combined)
spatial_txt = 'location'
fire_vio_df, fire_vio_parcel_df = spatial_join(fire_vio_df, 
                                                spatial_txt, 
                                                final_parcels_df)

In [22]:
fire_vio_parcel_df.head()

,inspection_number,violation_id,address,battalion,station,bfp_district,close_date,corrective_action,status,violation_item_description,...,:@computed_region_p5aj_wyqh,:@computed_region_jwn9_ihcz,:@computed_region_6qbp_sg9q,:@computed_region_qgnn_b9vv,:@computed_region_26cr_cadq,:@computed_region_6ezc_tdp2,:@computed_region_h4ep_8xdi,geometry,index_right,parcel_number
0,294593,1610-0045SFC13EY,9991 Unk Airport,AP,AP,AP,2022-12-26T00:00:00,correct - no permit,abated,2013 san francisco fire code,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-122.38159 37.62391),NaN,NaN
1,185687,1404-001213PER01Y,900 North Access Rd,AP,AP,AP,2014-04-07T00:00:00,correct - no permit,closed,permit / general,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-122.38636 37.63719),NaN,NaN
2,199268,1410-002613EXT04,400 Upper Domestic Loop,AP,AP,AP,2015-07-24T00:00:00,obtain permit,closed,extinguisher / service required,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-122.38503 37.61779),NaN,NaN
3,NaN,1706-013516SPR08Y,776 Bush St,01,02,01W,NaN,NaN,open,sprinkler / 5-year service,...,1,16,16,6,3,NaN,NaN,POINT (-122.41004 37.79026),13174.0,0273008
4,378677,1903-0278NOTES,1 Trenton St,01,02,01N,2019-04-30T09:14:39,NaN,abated,notes,...,1,104,104,6,3,NaN,NaN,POINT (-122.40908 37.79508),8656.0,0192006


In [12]:
#Check if we're duplicating data. This is looking OK, a few duplicates but nothing to worry too much about at this point
#There are a few points mapping to multiple parcels but we can take care of that later. If this was a longer project
#Could do some additional spatial logic to make sure mapping was only to one
print(fire_vio_df['inspection_number'].count())
print(fire_vio_df['inspection_number'].nunique())
print(fire_vio_parcel_df['inspection_number'].count())
print(fire_vio_parcel_df['inspection_number'].nunique())

38228
14134
39419
14134


In [13]:
# We're missing about 1% in the spatial join but at this level of analysis I think it is acceptable.
# If time permits, I will circle back and see why
fire_vio_parcel_df[fire_vio_parcel_df['parcel_number'].isna()]['inspection_number'].count()

460

In [14]:
#Drop these fields as postgres won't like and we have parcel now. 
fire_vio_parcel_df = fire_vio_parcel_df.drop(['location','geometry'],axis=1).drop_duplicates()

In [15]:
## Functionalize this for future...
#Preprocessing to make sure our date types transfer correctly
fire_vio_parcel_df['close_date'] = pd.to_datetime(fire_vio_parcel_df['close_date'], errors='coerce')
fire_vio_parcel_df['violation_date'] = pd.to_datetime(fire_vio_parcel_df['violation_date'], errors='coerce')
# Drop columns containing "@" in their names
columns_to_drop = fire_vio_parcel_df.filter(regex='@', axis=1).columns
fire_vio_parcel_df.drop(columns=columns_to_drop, inplace=True)

In [16]:
#Upload data to postgres database (running locally) to create extract and allow some joins to be done in SQL
tablename = 'fire_violations_raw'
upload_to_postgres(fire_vio_parcel_df, postgres_connstr, tablename)

219

## Import DBI Violations
- Download via API
- Join with parcels if possible, although DBI inspections data actually has block/lot number. Join anyway to check spatial joining methodology.
- Upload to postgres

In [23]:
filters = 'location IS NOT NULL' #going to start by filtering out data without a parcel because we cannot tell anything about building type without this if using parcel to join
pulltype = 'all'
dbi_violoations_endpoint = 'nbtm-fbw5'
dbi_vio_df = pull_opensf_data(site, 
                              dbi_violoations_endpoint, 
                              app_token, 
                              api_key_id, 
                              api_secret_key,
                              pulltype,
                              filters)

In [24]:
dbi_vio_df.head()

,complaint_number,item_sequence_number,date_filed,block,lot,street_number,street_name,street_suffix,status,receiving_division,assigned_division,nov_category_description,item,neighborhoods_analysis_boundaries,supervisor_district,zipcode,location,unit,nov_item_description
0,199705272,8127,1997-04-18T00:00:00.000,0281,017,1035,Bush,St,not active,Housing Inspection Services,Housing Inspection Services,other section,inspector comments,Nob Hill,3,94109,"{'type': 'Point', 'coordinates': [-122.4142531...",NaN,NaN
1,199919693,29804,1999-03-08T00:00:00.000,3643,015A,3356,24th,St,not active,Department Of Bldg Inspection,Housing Inspection Services,other section,inspector comments,Mission,9,94110,"{'type': 'Point', 'coordinates': [-122.4198301...",NaN,NaN
2,200447766,142256,2004-02-18T00:00:00.000,2800,006,899,Corbett,Av,not active,Housing Inspection Services,Housing Inspection Services,building section,unit 1,Twin Peaks,8,94131,"{'type': 'Point', 'coordinates': [-122.4434596...",NaN,NaN
3,200448900,144671,2004-03-30T00:00:00.000,3726,052,525,Natoma,St,active,Housing Inspection Services,Housing Inspection Services,other section,inspectors comments,South of Market,6,94103,"{'type': 'Point', 'coordinates': [-122.4081877...",NaN,NaN
4,200448376,143283,2004-03-11T00:00:00.000,0131,007,1309,Grant,Av,not active,Housing Inspection Services,Housing Inspection Services,building section,x,North Beach,3,94133,"{'type': 'Point', 'coordinates': [-122.4074470...",NaN,NaN


In [25]:
#Spatial join, although dbi inspections actually has block number and lot number (parcel number when combined)
spatial_txt = 'location'
dbi_vio_df, dbi_vio_parcel_df = spatial_join(dbi_vio_df, 
                                                spatial_txt, 
                                                final_parcels_df)

In [26]:
#This almost seems too good in terms of no missing parcels after the join. Checks confirm it is correct.
dbi_vio_parcel_df[dbi_vio_parcel_df['parcel_number'].isna()]['complaint_number'].count()

9

In [73]:
#Drop these fields as postgres won't like and we have parcel now. 
dbi_vio_parcel_df = dbi_vio_parcel_df.drop(['location','geometry'],axis=1).drop_duplicates()

In [90]:
## Functionalize this for future...
#Preprocessing to make sure our date types transfer correctly
dbi_vio_parcel_df['date_filed'] = pd.to_datetime(dbi_vio_parcel_df['date_filed'], errors='coerce')
# Drop columns containing "@" in their names
columns_to_drop = dbi_vio_parcel_df.filter(regex='@', axis=1).columns
dbi_vio_parcel_df.drop(columns=columns_to_drop, inplace=True)

In [92]:
#Upload data to postgres database (running locally) to create extract and allow some joins to be done in SQL
tablename = 'dbi_violations_raw'
upload_to_postgres(dbi_vio_parcel_df, postgres_connstr, tablename)

387

## Import DBI Inspection Complaints
- Download, join, pre-process, upload

In [110]:
#this dataset has parcel number baked in and we have it for most records!
#let's filter if parcel is missing and if it's not assigned to DBI (for now)
filters = 'parcel_number IS NOT NULL AND LOWER(assigned_division) = "housing inspection services"' 

pulltype = 'all'
dbi_inspec_endpoint = 'gm2e-bten'
dbi_inspec_df = pull_opensf_data(site, 
                              dbi_inspec_endpoint, 
                              app_token, 
                              api_key_id, 
                              api_secret_key,
                              pulltype,
                              filters)

In [111]:
#convert dates to date types
dbi_inspec_df['date_filed'] = pd.to_datetime(dbi_inspec_df['date_filed'], errors='coerce')
dbi_inspec_df['closed_date'] = pd.to_datetime(dbi_inspec_df['closed_date'], errors='coerce')
dbi_inspec_df['date_1st_nov_issued'] = pd.to_datetime(dbi_inspec_df['date_1st_nov_issued'], errors='coerce')
dbi_inspec_df['date_abated'] = pd.to_datetime(dbi_inspec_df['date_abated'], errors='coerce')
dbi_inspec_df['date_2nd_nov_issued'] = pd.to_datetime(dbi_inspec_df['date_2nd_nov_issued'], errors='coerce')
dbi_inspec_df['date_referred_to_city_attorney'] = pd.to_datetime(dbi_inspec_df['date_referred_to_city_attorney'], errors='coerce')
dbi_inspec_df['date_final_warning_letter_issued'] = pd.to_datetime(dbi_inspec_df['date_final_warning_letter_issued'], errors='coerce')
dbi_inspec_df['director_hearing_date'] = pd.to_datetime(dbi_inspec_df['director_hearing_date'], errors='coerce')

In [112]:
dbi_inspec_df.drop(['point'],axis=1,inplace=True)

In [113]:
#Upload data to postgres database (running locally) to create extract and allow some joins to be done in SQL
tablename = 'dbi_inspections_raw'
upload_to_postgres(dbi_inspec_df, postgres_connstr, tablename)

474

## Next Steps
- Now that data has been extracted from API and loaded into SQL, we will investigate data further in pgAdmin and write transformations in SQL to join data, feature engineer, etc.
- In production, our SQL transformations would be managed by something like dbt but for the purposes of this project that has not been set-up.